In [1]:
import polars as pl
pl.Config.set_tbl_rows(40)

import numpy as np
from scipy.stats import norm
from scipy.special import ndtr

In [ ]:

date_str = '20250829'

df = pl.read_csv('temp.csv')
df = df.with_columns(
    [(pl.col(col) * pl.col('S')).round(1).alias(col+'_usd') for col in ['bid_price','ask_price','bid_price_P','ask_price_P']]
)

In [ ]:
pl.read_csv(f'{date_str}/btc')

In [3]:
df.tail(5)

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.013,0.0135,117000,0.063,0.075582,"""19SEP25""",110106.62,0.057078,110618.96,1431.4,1486.4,6936.7,8322.1
"""2025-08-29T12:00:00.000000""",0.011,0.0115,118000,0.063,0.084664,"""19SEP25""",110106.62,0.057078,110618.96,1211.2,1266.2,6936.7,9322.1
"""2025-08-29T12:00:00.000000""",0.008,0.0085,120000,0.063,0.102828,"""19SEP25""",110106.62,0.057078,110618.96,880.9,935.9,6936.7,11322.1
"""2025-08-29T12:00:00.000000""",0.0035,0.0039,125000,0.095,0.148239,"""19SEP25""",110106.62,0.057078,110618.96,385.4,429.4,10460.1,16322.1
"""2025-08-29T12:00:00.000000""",0.0017,0.0021,130000,0.137,0.193649,"""19SEP25""",110106.62,0.057078,110618.96,187.2,231.2,15084.6,21322.1


In [ ]:


N = ndtr  # Normal CDF function
# N = norm.cdf

def black76_call(F, K, T, r, vol):
    """
    Black-76 call option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price 
    T: Time to expiration (years)
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (F * N(d1) - K * N(d2))

def black76_put(F, K, T, r, vol):
    """
    Black-76 put option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years) 
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (K * N(-d2) - F * N(-d1))

def black76_vega(F, K, T, r, sigma):
    """
    Black-76 vega (sensitivity to volatility) for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years)
    r: Risk-free rate  
    sigma: Volatility
    """
    d1 = (np.log(F / K) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    return np.exp(-r * T) * F * norm.pdf(d1) * np.sqrt(T)

# Keep aliases for backward compatibility
bs_call = black76_call
bs_put = black76_put
bs_vega = black76_vega

In [7]:
class Black76OptionPricer:
    """
    Black-76 option pricing model for options on forward contracts.
    
    This class provides pricing and Greeks calculation for European options
    on forward contracts using the Black-76 model.
    
    Key Features:
    - Call and Put option pricing
    - All Greeks: Delta, Gamma, Theta, Vega, Rho
    - Forward Delta and Gamma
    - Vectorized operations for efficient computation
    - Implied volatility calculation using Newton-Raphson
    """
    
    def __init__(self, F, K, T, r, sigma):
        """
        Initialize the Black-76 option pricer.
        
        Parameters:
        -----------
        F : float or array-like
            Forward price
        K : float or array-like
            Strike price
        T : float or array-like
            Time to expiration (in years)
        r : float or array-like
            Risk-free interest rate (for discounting)
        sigma : float or array-like
            Volatility (annualized)
        """
        self.F = np.asarray(F, dtype=float)
        self.K = np.asarray(K, dtype=float)
        self.T = np.asarray(T, dtype=float)
        self.r = np.asarray(r, dtype=float)
        self.sigma = np.asarray(sigma, dtype=float)
        
        # Calculate d1 and d2 once for efficiency
        self._calculate_d_params()
    
    def _calculate_d_params(self):
        """Calculate d1 and d2 parameters used in Black-76 formulas."""
        sqrt_T = np.sqrt(self.T)
        sigma_sqrt_T = self.sigma * sqrt_T
        
        self.d1 = (np.log(self.F / self.K) + 0.5 * self.sigma**2 * self.T) / sigma_sqrt_T
        self.d2 = self.d1 - sigma_sqrt_T
        
        # Cache normal distribution values for efficiency
        self.N_d1 = ndtr(self.d1)
        self.N_d2 = ndtr(self.d2)
        self.N_neg_d1 = ndtr(-self.d1)
        self.N_neg_d2 = ndtr(-self.d2)
        
        # Cache normal density values
        self.n_d1 = norm.pdf(self.d1)
        self.n_d2 = norm.pdf(self.d2)
    
    def call_price(self):
        """
        Calculate Black-76 call option price.
        
        Returns:
        --------
        float or numpy.ndarray
            Call option price(s)
        """
        discount_factor = np.exp(-self.r * self.T)
        return discount_factor * (self.F * self.N_d1 - self.K * self.N_d2)
    
    def put_price(self):
        """
        Calculate Black-76 put option price.
        
        Returns:
        --------
        float or numpy.ndarray
            Put option price(s)
        """
        discount_factor = np.exp(-self.r * self.T)
        return discount_factor * (self.K * self.N_neg_d2 - self.F * self.N_neg_d1)
    
    def call_delta(self):
        """
        Calculate call option delta (∂C/∂S).
        Note: For forwards, this is the sensitivity to the underlying spot price.
        
        Returns:
        --------
        float or numpy.ndarray
            Call delta
        """
        return np.exp(-self.r * self.T) * self.N_d1
    
    def put_delta(self):
        """
        Calculate put option delta (∂P/∂S).
        
        Returns:
        --------
        float or numpy.ndarray
            Put delta
        """
        return -np.exp(-self.r * self.T) * self.N_neg_d1
    
    def gamma(self):
        """
        Calculate option gamma (∂²V/∂S²).
        Same for calls and puts.
        
        Returns:
        --------
        float or numpy.ndarray
            Option gamma
        """
        discount_factor = np.exp(-self.r * self.T)
        return discount_factor * self.n_d1 / (self.F * self.sigma * np.sqrt(self.T))
    
    def vega(self):
        """
        Calculate option vega (∂V/∂σ).
        Same for calls and puts.
        
        Returns:
        --------
        float or numpy.ndarray
            Option vega
        """
        discount_factor = np.exp(-self.r * self.T)
        return discount_factor * self.F * self.n_d1 * np.sqrt(self.T)
    
    def call_theta(self):
        """
        Calculate call option theta (∂C/∂T).
        
        Returns:
        --------
        float or numpy.ndarray
            Call theta (negative for time decay)
        """
        discount_factor = np.exp(-self.r * self.T)
        sqrt_T = np.sqrt(self.T)
        
        term1 = -discount_factor * self.F * self.n_d1 * self.sigma / (2 * sqrt_T)
        term2 = self.r * discount_factor * (self.F * self.N_d1 - self.K * self.N_d2)
        
        return term1 + term2
    
    def put_theta(self):
        """
        Calculate put option theta (∂P/∂T).
        
        Returns:
        --------
        float or numpy.ndarray
            Put theta (negative for time decay)
        """
        discount_factor = np.exp(-self.r * self.T)
        sqrt_T = np.sqrt(self.T)
        
        term1 = -discount_factor * self.F * self.n_d1 * self.sigma / (2 * sqrt_T)
        term2 = self.r * discount_factor * (self.K * self.N_neg_d2 - self.F * self.N_neg_d1)
        
        return term1 + term2
    
    def call_rho(self):
        """
        Calculate call option rho (∂C/∂r).
        
        Returns:
        --------
        float or numpy.ndarray
            Call rho
        """
        call_price = self.call_price()
        return -self.T * call_price
    
    def put_rho(self):
        """
        Calculate put option rho (∂P/∂r).
        
        Returns:
        --------
        float or numpy.ndarray
            Put rho
        """
        put_price = self.put_price()
        return -self.T * put_price
    
    def get_all_greeks(self, option_type='call'):
        """
        Get all Greeks for the specified option type.
        
        Parameters:
        -----------
        option_type : str
            'call' or 'put'
        
        Returns:
        --------
        dict
            Dictionary containing all Greeks and option price
        """
        if option_type.lower() == 'call':
            return {
                'price': self.call_price(),
                'delta': self.call_delta(),
                'gamma': self.gamma(),
                'theta': self.call_theta(),
                'vega': self.vega(),
                'rho': self.call_rho()
            }
        elif option_type.lower() == 'put':
            return {
                'price': self.put_price(),
                'delta': self.put_delta(),
                'gamma': self.gamma(),
                'theta': self.put_theta(),
                'vega': self.vega(),
                'rho': self.put_rho()
            }
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    
    def implied_volatility(self, target_price, option_type='call', max_iterations=200, precision=1e-5):
        """
        Calculate implied volatility using Newton-Raphson method.
        
        Parameters:
        -----------
        target_price : float or array-like
            Market price of the option
        option_type : str
            'call' or 'put'
        max_iterations : int
            Maximum number of iterations
        precision : float
            Convergence precision
        
        Returns:
        --------
        float or numpy.ndarray
            Implied volatility
        """
        target_price = np.asarray(target_price, dtype=float)
        
        # Determine if we have scalar or array inputs
        is_scalar = target_price.ndim == 0
        
        # Ensure we work with arrays internally
        if is_scalar:
            target_price = np.array([target_price])
        
        # Initial guess for volatility
        sigma_guess = np.full_like(target_price, 0.5, dtype=float)
        
        for i in range(max_iterations):
            # Create new pricer with current volatility guess
            temp_pricer = Black76OptionPricer(self.F, self.K, self.T, self.r, sigma_guess)
            
            # Calculate price and vega
            if option_type.lower() == 'call':
                price = temp_pricer.call_price()
            else:
                price = temp_pricer.put_price()
            
            vega = temp_pricer.vega()
            
            # Calculate price difference
            diff = target_price - price
            
            # Check convergence
            if np.all(np.abs(diff) < precision):
                break
            
            # Newton-Raphson update (only where vega is not zero)
            mask = (vega != 0) & np.isfinite(vega) & np.isfinite(diff)
            sigma_guess = np.where(mask, sigma_guess + diff / vega, sigma_guess)
            
            # Ensure volatility stays positive
            sigma_guess = np.maximum(sigma_guess, 1e-6)
        
        # Return scalar if input was scalar, otherwise return array
        return float(sigma_guess[0]) if is_scalar else sigma_guess
    
    def update_parameters(self, **kwargs):
        """
        Update one or more parameters and recalculate d1, d2.
        
        Parameters:
        -----------
        **kwargs : dict
            Parameters to update (F, K, T, r, sigma)
        """
        for param, value in kwargs.items():
            if hasattr(self, param):
                setattr(self, param, np.asarray(value, dtype=float))
            else:
                raise ValueError(f"Invalid parameter: {param}")
        
        # Recalculate d parameters
        self._calculate_d_params()
    
    def __repr__(self):
        """String representation of the pricer."""
        return (f"Black76OptionPricer(F={self.F}, K={self.K}, T={self.T}, "
                f"r={self.r}, sigma={self.sigma})")

# Convenience functions for backward compatibility
def black76_call(F, K, T, r, vol):
    """Convenience function for Black-76 call pricing."""
    pricer = Black76OptionPricer(F, K, T, r, vol)
    return pricer.call_price()

def black76_put(F, K, T, r, vol):
    """Convenience function for Black-76 put pricing."""
    pricer = Black76OptionPricer(F, K, T, r, vol)
    return pricer.put_price()

def black76_vega(F, K, T, r, sigma):
    """Convenience function for Black-76 vega calculation."""
    pricer = Black76OptionPricer(F, K, T, r, sigma)
    return pricer.vega()

In [10]:
# Example usage of the Black76OptionPricer class
print("🚀 Black-76 Option Pricing Demo")
print("=" * 50)

# Example parameters
F = 100.0      # Forward price
K = 105.0      # Strike price
T = 0.25       # Time to expiration (3 months)
r = 0.05       # Risk-free rate (5%)
sigma = 0.250   # Volatility (25%)

# Create the pricer
pricer = Black76OptionPricer(F, K, T, r, sigma)
print(f"Parameters: F=${F}, K=${K}, T={T:.2f}y, r={r:.1%}, σ={sigma:.1%}")
print()

# Get call option pricing and Greeks
call_greeks = pricer.get_all_greeks('call')
print("📈 CALL OPTION:")
print(f"  Price:         ${call_greeks['price']:.4f}")
print(f"  Delta:         {call_greeks['delta']:.4f}")
print(f"  Gamma:         {call_greeks['gamma']:.6f}")
print(f"  Theta:         {call_greeks['theta']:.4f}")
print(f"  Vega:          {call_greeks['vega']:.4f}")
print(f"  Rho:           {call_greeks['rho']:.4f}")
print()

# Get put option pricing and Greeks
put_greeks = pricer.get_all_greeks('put')
print("📉 PUT OPTION:")
print(f"  Price:         ${put_greeks['price']:.4f}")
print(f"  Delta:         {put_greeks['delta']:.4f}")
print(f"  Gamma:         {put_greeks['gamma']:.6f}")
print(f"  Theta:         {put_greeks['theta']:.4f}")
print(f"  Vega:          {put_greeks['vega']:.4f}")
print(f"  Rho:           {put_greeks['rho']:.4f}")
print()

# Verify put-call parity: C - P = e^(-rT) * (F - K)
parity_check = call_greeks['price'] - put_greeks['price']
theoretical_parity = np.exp(-r * T) * (F - K)
print("🔍 PUT-CALL PARITY CHECK:")
print(f"  C - P = ${parity_check:.6f}")
print(f"  e^(-rT)*(F-K) = ${theoretical_parity:.6f}")
print(f"  Difference: ${abs(parity_check - theoretical_parity):.8f}")
print()

# Implied volatility demonstration
print("🎯 IMPLIED VOLATILITY TEST:")
market_call_price = call_greeks['price']
implied_vol = pricer.implied_volatility(market_call_price, 'call')
print(f"  Original volatility: {sigma:.1%}")
print(f"  Implied volatility:  {implied_vol:.1%}")
print(f"  Difference:          {abs(sigma - implied_vol):.8f}")

🚀 Black-76 Option Pricing Demo
Parameters: F=$100.0, K=$105.0, T=0.25y, r=5.0%, σ=25.0%

📈 CALL OPTION:
  Price:         $2.9546
  Delta:         0.3669
  Gamma:         0.029870
  Theta:         -9.1866
  Vega:          18.6688
  Rho:           -0.7387

📉 PUT OPTION:
  Price:         $7.8925
  Delta:         -0.6207
  Gamma:         0.029870
  Theta:         -8.9398
  Vega:          18.6688
  Rho:           -1.9731

🔍 PUT-CALL PARITY CHECK:
  C - P = $-4.937889
  e^(-rT)*(F-K) = $-4.937889
  Difference: $0.00000000

🎯 IMPLIED VOLATILITY TEST:
  Original volatility: 25.0%
  Implied volatility:  25.0%
  Difference:          0.00000000


In [9]:
# Vectorized operations example - multiple strikes
print("🔢 VECTORIZED OPERATIONS DEMO")
print("=" * 50)

# Multiple strikes for volatility smile analysis
strikes = np.array([80, 85, 90, 95, 100, 105, 110, 115, 120])
F_vec = 100.0  # Single forward price
T_vec = 0.25   # Single time to expiration
r_vec = 0.05   # Single risk-free rate
sigma_vec = 0.25  # Single volatility

# Create vectorized pricer
vec_pricer = Black76OptionPricer(F_vec, strikes, T_vec, r_vec, sigma_vec)

# Calculate call prices and Greeks for all strikes
call_prices = vec_pricer.call_price()
call_deltas = vec_pricer.call_delta()
call_gammas = vec_pricer.gamma()
call_vegas = vec_pricer.vega()

# Create a DataFrame for nice display
import polars as pl

option_chain = pl.DataFrame({
    'Strike': strikes,
    'Call_Price': call_prices.round(4),
    'Call_Delta': call_deltas.round(4),
    'Gamma': call_gammas.round(6),
    'Vega': call_vegas.round(4),
    'Moneyness': (strikes / F_vec).round(3)
})

print("📊 OPTION CHAIN (Call Options):")
print(option_chain)
print()

# Calculate implied volatilities from the prices
print("🎯 IMPLIED VOLATILITY ROUND-TRIP TEST:")
implied_vols = vec_pricer.implied_volatility(call_prices, 'call')
print(f"Original volatility: {sigma_vec:.1%}")
print(f"Max implied vol diff: {np.max(np.abs(implied_vols - sigma_vec)):.8f}")
print(f"All implied vols within tolerance: {np.allclose(implied_vols, sigma_vec, atol=1e-6)}")
print()

# Demonstrate parameter updating
print("🔄 DYNAMIC PARAMETER UPDATE:")
print("Updating volatility from 25% to 30%...")
vec_pricer.update_parameters(sigma=0.30)
new_call_prices = vec_pricer.call_price()
price_change = new_call_prices - call_prices

price_comparison = pl.DataFrame({
    'Strike': strikes,
    'Old_Price_25%': call_prices.round(4),
    'New_Price_30%': new_call_prices.round(4),
    'Price_Change': price_change.round(4),
    'Pct_Change': (price_change / call_prices * 100).round(2)
})

print(price_comparison)

🔢 VECTORIZED OPERATIONS DEMO
📊 OPTION CHAIN (Call Options):
shape: (9, 6)
┌────────┬────────────┬────────────┬──────────┬─────────┬───────────┐
│ Strike ┆ Call_Price ┆ Call_Delta ┆ Gamma    ┆ Vega    ┆ Moneyness │
│ ---    ┆ ---        ┆ ---        ┆ ---      ┆ ---     ┆ ---       │
│ i64    ┆ f64        ┆ f64        ┆ f64      ┆ f64     ┆ f64       │
╞════════╪════════════╪════════════╪══════════╪═════════╪═══════════╡
│ 80     ┆ 19.9149    ┆ 0.9557     ┆ 0.005718 ┆ 3.574   ┆ 0.8       │
│ 85     ┆ 15.331     ┆ 0.9022     ┆ 0.012456 ┆ 7.7848  ┆ 0.85      │
│ 90     ┆ 11.1787    ┆ 0.8072     ┆ 0.02092  ┆ 13.0753 ┆ 0.9       │
│ 95     ┆ 7.6653     ┆ 0.6734     ┆ 0.028185 ┆ 17.6157 ┆ 0.95      │
│ 100    ┆ 4.9216     ┆ 0.5184     ┆ 0.031457 ┆ 19.6609 ┆ 1.0       │
│ 105    ┆ 2.9546     ┆ 0.3669     ┆ 0.02987  ┆ 18.6688 ┆ 1.05      │
│ 110    ┆ 1.66       ┆ 0.239      ┆ 0.02467  ┆ 15.419  ┆ 1.1       │
│ 115    ┆ 0.8752     ┆ 0.1438     ┆ 0.018055 ┆ 11.2847 ┆ 1.15      │
│ 120    ┆ 0.434

In [5]:
def find_vol(target_value, F, K, T, r, **kwargs):
    """
    Vectorized implied volatility solver using Newton-Raphson method with Black-76 model.
    
    All arguments can be numpy arrays, scalars, or Polars expressions.
    Returns array of implied volatilities or scalar if all inputs are scalar.
    
    Parameters:
    -----------
    target_value : array-like or scalar
        Option market prices
    F : array-like or scalar  
        Forward prices
    K : array-like or scalar
        Strike prices
    T : array-like or scalar
        Time to expiration (in years)
    r : array-like or scalar
        Risk-free interest rate (for discounting)
    option_type : str, optional
        'C' for call (default), 'P' for put
    
    Returns:
    --------
    numpy.ndarray or float
        Implied volatilities
    """
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    
    # Convert all inputs to numpy arrays for vectorization
    target_value = np.asarray(target_value, dtype=float)
    F = np.asarray(F, dtype=float)
    K = np.asarray(K, dtype=float)
    T = np.asarray(T, dtype=float)
    r = np.asarray(r, dtype=float)
    option_type = kwargs.get('option_type', 'C')
    
    # Get the broadcasted shape
    try:
        shape = np.broadcast(target_value, F, K, T, r).shape
    except ValueError as e:
        raise ValueError(f"Input arrays could not be broadcast together: {e}")
    
    # Initialize volatility array
    sigmas = np.full(shape, 0.5, dtype=float)
    done = np.zeros(shape, dtype=bool)
    
    # Newton-Raphson iterations
    for iteration in range(MAX_ITERATIONS):
        # Calculate option prices and vegas using Black-76
        if option_type == 'P':
            prices = black76_put(F, K, T, r, sigmas)
        else:
            prices = black76_call(F, K, T, r, sigmas)
        
        vegas = black76_vega(F, K, T, r, sigmas)
        
        # Calculate price differences
        diff = target_value - prices
        
        # Check convergence
        converged = np.abs(diff) < PRECISION
        
        # Only update where not converged and vega is not zero
        update_mask = ~done & (vegas != 0) & np.isfinite(vegas) & np.isfinite(diff)
        
        # Newton-Raphson update
        sigmas[update_mask] += diff[update_mask] / vegas[update_mask]
        
        # Ensure volatility stays positive
        sigmas = np.maximum(sigmas, 1e-6)
        
        # Mark converged elements as done
        done = done | converged
        
        # Early exit if all converged
        if np.all(done):
            break
    
    # Return scalar if input was scalar, otherwise return array
    if sigmas.shape == ():
        return float(sigmas)
    else:
        return sigmas

In [6]:
df_vola =\
df.with_columns(
    call_bid_vola = find_vol(
        target_value=df['bid_price_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=(interest_rate:=0.08),
        option_type='C'
    ).round(4),
    call_ask_vola = find_vol(
        target_value=df['ask_price_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(), 
        r=interest_rate,
        option_type='C'
    ).round(4),
    put_bid_vola = find_vol(
        target_value=df['bid_price_P_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4),
    put_ask_vola = find_vol(
        target_value=df['ask_price_P_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4)
)

In [7]:
df_vola

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd,call_bid_vola,call_ask_vola,put_bid_vola,put_ask_vola
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.1125,0.156739,95000,0.004,0.0043,"""19SEP25""",110106.62,0.057078,110618.96,12387.0,17258.0,440.4,473.5,0.0,0.7049,0.4642,0.4728
"""2025-08-29T12:00:00.000000""",0.087,0.129493,98000,0.0055,0.0065,"""19SEP25""",110106.62,0.057078,110618.96,9579.3,14258.0,605.6,715.7,0.0,0.6073,0.4265,0.4483
"""2025-08-29T12:00:00.000000""",0.073,0.111328,100000,0.0075,0.0085,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,12258.0,825.8,935.9,0.0,0.542,0.4134,0.4316
"""2025-08-29T12:00:00.000000""",0.073,0.093164,102000,0.0105,0.011,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,10258.0,1156.1,1211.2,0.0,0.476,0.4061,0.4138
"""2025-08-29T12:00:00.000000""",0.073,0.075,104000,0.014,0.0145,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,8258.0,1541.5,1596.5,0.382,0.4089,0.3934,0.4001
"""2025-08-29T12:00:00.000000""",0.066,0.068,105000,0.016,0.0165,"""19SEP25""",110106.62,0.057078,110618.96,7267.0,7487.3,1761.7,1816.8,0.3756,0.4008,0.3857,0.3921
"""2025-08-29T12:00:00.000000""",0.0595,0.0615,106000,0.0185,0.019,"""19SEP25""",110106.62,0.057078,110618.96,6551.3,6771.6,2037.0,2092.0,0.3719,0.3959,0.3811,0.387
"""2025-08-29T12:00:00.000000""",0.053,0.0555,107000,0.021,0.022,"""19SEP25""",110106.62,0.057078,110618.96,5835.7,6110.9,2312.2,2422.3,0.3647,0.3932,0.3729,0.3843
"""2025-08-29T12:00:00.000000""",0.0475,0.049,108000,0.024,0.025,"""19SEP25""",110106.62,0.057078,110618.96,5230.1,5395.2,2642.6,2752.7,0.3651,0.3816,0.367,0.378


In [8]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_bid_vola'], mode='markers', name='Call Bid Vola', marker=dict(symbol='triangle-up', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_ask_vola'], mode='markers', name='Call Ask Vola', marker=dict(symbol='triangle-down', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_bid_vola'], mode='markers', name='Put Bid Vola', marker=dict(symbol='triangle-up', color='red')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_ask_vola'], mode='markers', name='Put Ask Vola', marker=dict(symbol='triangle-down', color='red')))
fig.add_vline(x=df_vola['S'][0], line_color="black", annotation_text="Spot (S)", annotation_position="top")
fig.update_layout(
    title='Implied Volatility vs Strike Price',
    xaxis_title='Strike Price',
    yaxis_title='Implied Volatility',
    yaxis=dict(range=[0.2, 0.8])  # Set the y-axis range as needed
)
fig.show()

In [20]:
date_str = "20240229"  # Fixed: removed extra digit
df = pl.read_csv('sample_data_2.csv').with_columns(
            timestamp=(pl.lit(date_str) + " " + pl.col("time")).str.strptime(pl.Datetime('ns'), "%Y%m%d %H:%M:%S%.f"),
        ).drop('time')
df.head()

data,expiry,spot,time_to_expiry,strike,pminusc,weight,timestamp
str,i64,f64,f64,f64,f64,f64,datetime[ns]
"""Data""",20240301,62480.71,0.003653,48000.0,-14492.400685,0.000012,2024-02-29 00:00:06
"""Data""",20240301,62480.71,0.003653,49000.0,-13489.585289,0.000011,2024-02-29 00:00:06
"""Data""",20240301,62480.71,0.003653,50000.0,-12474.273752,0.000015,2024-02-29 00:00:06
"""Data""",20240301,62480.71,0.003653,50500.0,-11971.304036,0.000015,2024-02-29 00:00:06
"""Data""",20240301,62480.71,0.003653,51000.0,-11452.714143,0.000011,2024-02-29 00:00:06


In [21]:
from datetime import datetime

df.filter(pl.col('timestamp')==datetime(2024,2,29,0,5))

data,expiry,spot,time_to_expiry,strike,pminusc,weight,timestamp
str,i64,f64,f64,f64,f64,f64,datetime[ns]
"""Data""",20240301,62192.34,0.003643,48000.0,-14192.291988,0.000015,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,49000.0,-13178.556846,0.000018,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,50000.0,-12214.575576,0.00002,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,50500.0,-11713.927239,0.000019,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,51000.0,-11179.073115,0.000019,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,51500.0,-10712.630565,0.000019,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,52000.0,-10162.228356,0.000015,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,52500.0,-9714.443508,0.000018,2024-02-29 00:05:00
"""Data""",20240301,62192.34,0.003643,53000.0,-9195.137469,0.000014,2024-02-29 00:05:00
